In [25]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
n_samples=50000
n_features=20
X, y = make_classification(n_samples=n_samples, n_features=n_features, n_informative=5, n_redundant=2, n_repeated=0, n_classes=2,\
                           n_clusters_per_class=2, weights=[0.9,0.1], flip_y=0.01, class_sep=1.0, hypercube=True, shift=0.0,\
                           scale=1.0, shuffle=True, random_state=None)
y=np.array(y)
X=np.array(X)
y=np.reshape(y,(n_samples, 1))
X=np.reshape(X, (n_samples, n_features))
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)


In [32]:
from tensorflow.examples.tutorials.mnist import input_data
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow-gpu as tf

# Use Tenserflow backend
sess = tf.Session()
K.set_session(sess)

batch_size = 32
n_z = 5
n_epoch = 10


# Q(z|X) -- encoder
inputs = Input(shape=(20,))
h_q = Dense(15, activation='relu')(inputs)
mu = Dense(n_z, activation='linear')(h_q)
log_sigma = Dense(n_z, activation='linear')(h_q)

In [33]:
def sample_z(args):
    mu, log_sigma = args
    eps = K.random_normal(shape=(batch_size, n_z), mean=0., stddev=1.)
    return mu + K.exp(log_sigma / 2) * eps


# Sample z ~ Q(z|X)
z = Lambda(sample_z)([mu, log_sigma])

In [34]:
# P(X|z) -- decoder
decoder_hidden = Dense(15, activation='relu')
decoder_out = Dense(20, activation='sigmoid')

h_p = decoder_hidden(z)
outputs = decoder_out(h_p)

In [35]:
# Overall VAE model, for reconstruction and training
vae = Model(inputs, outputs)

# Encoder model, to encode input into latent variable
# We use the mean as the output as it is the center point, the representative of the gaussian
encoder = Model(inputs, mu)

# Generator model, generate new data given latent variable z
d_in = Input(shape=(n_z,))
d_h = decoder_hidden(d_in)
d_out = decoder_out(d_h)
decoder = Model(d_in, d_out)

In [36]:
def vae_loss(y_true, y_pred):
    """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
    # E[log P(X|z)]
    recon = K.sum(K.binary_crossentropy(y_pred, y_true), axis=1)
    # D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
    kl = 0.5 * K.sum(K.exp(log_sigma) + K.square(mu) - 1. - log_sigma, axis=1)

    return recon + kl

In [37]:
vae.compile(optimizer='adam', loss=vae_loss)
vae.fit(X_train, X_train, batch_size=batch_size, epochs=n_epoch)

Epoch 1/10
33152/33500 [============================>.] - ETA: 0s - loss: 69.2737    

InvalidArgumentError: Incompatible shapes: [32] vs. [28]
	 [[Node: training_5/Adam/gradients/loss_5/dense_27_loss/mul_1_grad/BroadcastGradientArgs = BroadcastGradientArgs[T=DT_INT32, _class=["loc:@loss_5/dense_27_loss/mul_1"], _device="/job:localhost/replica:0/task:0/cpu:0"](training_5/Adam/gradients/loss_5/dense_27_loss/mul_1_grad/Shape, training_5/Adam/gradients/loss_5/dense_27_loss/mul_1_grad/Shape_1)]]

Caused by op 'training_5/Adam/gradients/loss_5/dense_27_loss/mul_1_grad/BroadcastGradientArgs', defined at:
  File "C:\Users\Snir\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Snir\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Snir\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\Snir\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Snir\Ana